In [68]:
import subprocess
import shlex
import datetime
from dateutil.tz import tzlocal
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt 
import numpy as np 
import time
import glob

def run_command_async(command):
    if (isinstance(command, list) == False):
        commands = [command]
    else:
        commands=command
    processes = [None] * len(commands)
    for i in range(len(commands)):
        processes[i] = subprocess.Popen(shlex.split(commands[i]), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    return processes

def wait_for_command_async(processes):
    while processes:
        for i, process in enumerate(processes):
            output = process.stdout.readline().decode()
            if output == '' and process.poll() is not None:
                processes.remove(process)
                break
            if output:
                now=datetime.datetime.now(tzlocal())
                strnow = now.strftime("%Y-%m-%d %H:%M:%S")
                print ("Log {0} - {1} : ".format(i,strnow) + output.strip())
    
    rc = process.poll()
    return rc    
def run_command(command, shell=False):
    print(command)
    if (isinstance(command, list) == False):
        commands = [command]
    else:
        commands=command
    processes = [None] * len(commands)
    for i in range(len(commands)):
        print (commands[i])
        processes[i] = subprocess.Popen(shlex.split(commands[i]), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=shell)
    while processes:
        for i, process in enumerate(processes):
            output = process.stdout.readline().decode()
            if output == '' and process.poll() is not None:
                processes.remove(process)
                break
            if output:
                now=datetime.datetime.now(tzlocal())
                strnow = now.strftime("%Y-%m-%d %H:%M:%S")
                print ("Log {0} - {1} : ".format(i,strnow) + output.strip())
    
    rc = process.poll()
    return rc
import os

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [69]:

def get_df_process(host, tenant=None):
    port=5434
    login="postgres"
    password="migrationTestDb_18"
    dbname="postgres"
#    login="bonitauser"
#    password="migrationTestDb_18"
#    dbname="bonitadb"
    conn = psycopg2.connect("host = '{0}' dbname = '{1}' user = '{2}' password = '{3}' port = {4}".
             format(host, dbname, login, password, port))
    if tenant is None:
        sql = "select api_id.sourceobjectid as id, api_begin.startdate as start, api_state1.startdate as ready, api_end.enddate as end\n"\
        ", afi_begin.sourceobjectid as task_id, afi_begin.reachedstatedate as task_start\n"\
        ", afi_assign.reachedstatedate as task_ready, afi_assign.claimeddate as task_assign, afi_end.reachedstatedate as task_end\n"\
        "from (\n"\
        "	select distinct sourceobjectid\n"\
        "	from arch_process_instance api) as api_id\n"\
        "join arch_process_instance api_begin on api_id.sourceobjectid = api_begin.sourceobjectid and api_begin.stateid = 0\n"\
        "left join arch_process_instance api_end on api_id.sourceobjectid = api_end.sourceobjectid and api_end.stateid = 6\n"\
        "left join arch_process_instance api_state1 on api_id.sourceobjectid = api_state1.sourceobjectid and api_state1.stateid = 1\n"\
        "left join arch_flownode_instance afi_begin on api_id.sourceobjectid = afi_begin.parentcontainerid and afi_begin.stateid = 32 and afi_begin.kind = 'user'\n"\
        "left join arch_flownode_instance afi_end on afi_begin.sourceobjectid = afi_end.sourceobjectid and afi_end.stateid = 2 and afi_end.kind = 'user'\n"\
        "left join arch_flownode_instance afi_assign on afi_begin.sourceobjectid = afi_assign.sourceobjectid and afi_assign.stateid = 4 and afi_assign.kind = 'user' \n"\
        "order by id,task_id "
    else:
        cur = conn.cursor()
        cur.execute("select id from tenant where name = '{0}'".format(tenant))
        res = cur.fetchone()
        if res == None:
            return None
        tenantid = res[0]
        sql = "select api_id.sourceobjectid as id, api_begin.startdate as start, api_state1.startdate as ready, api_end.enddate as end\n"\
        ", afi_begin.sourceobjectid as task_id, afi_begin.reachedstatedate as task_start\n"\
        ", afi_assign.reachedstatedate as task_ready, afi_assign.claimeddate as task_assign, afi_end.reachedstatedate as task_end\n"\
        "from (\n"\
        "	select distinct sourceobjectid\n"\
        "	from arch_process_instance api where tenantid = {0}) as api_id\n"\
        "join arch_process_instance api_begin on api_id.sourceobjectid = api_begin.sourceobjectid and api_begin.tenantid={0} and api_begin.stateid = 0\n"\
        "left join arch_process_instance api_end on api_id.sourceobjectid = api_end.sourceobjectid and api_end.tenantid={0} and api_end.stateid = 6\n"\
        "left join arch_process_instance api_state1 on api_id.sourceobjectid = api_state1.sourceobjectid and api_state1.tenantid={0} and api_state1.stateid = 1\n"\
        "left join arch_flownode_instance afi_begin on api_id.sourceobjectid = afi_begin.parentcontainerid and afi_begin.tenantid={0} and afi_begin.stateid = 32 and afi_begin.kind = 'user'\n"\
        "left join arch_flownode_instance afi_end on afi_begin.sourceobjectid = afi_end.sourceobjectid and afi_end.tenantid={0} and afi_end.stateid = 2 and afi_end.kind = 'user'\n"\
        "left join arch_flownode_instance afi_assign on afi_begin.sourceobjectid = afi_assign.sourceobjectid and afi_assign.tenantid={0} and afi_assign.stateid = 4 and afi_assign.kind = 'user' \n"\
        "order by id,task_id ".format(tenantid)
        
    cur = conn.cursor()
    cur.execute(sql)
    df = pd.DataFrame(cur.fetchall(), columns=["id", "start", "ready", "end", "task_id", "task_start", "task_ready", "task_assign", "task_end"])
    df["duration"] = df["end"] - df["start"]
    df["start"] = pd.to_datetime(df["start"]/1000, unit="s")
    df["end"] = pd.to_datetime(df["end"]/1000, unit="s")

    return df

In [70]:
def launch_tenant(type_, migration_result_name, bpm, dur, tenants, targets, nb_agent, faban_ips, bpm_ips, tenants_migr=None, duration_migr=None, back_and_forth=None, duration_before_migration=30):
    if back_and_forth is None:
        back_and_forth = 0
    with cd ("./test_executor"):
        if tenants_migr is None:
            tenants_migr = []
        
        # prepare launch for every tenant
        for i in range(len(tenants)):  
            if tenants[i] in tenants_migr:
                if duration_migr > 0:
                    command = ["./benchflow_onlytest_xp3.sh generate_test {0} {1} 5 {2} 0 {3} {4} {5} {6} {7}".format(
                        nb_agent,
                        bpm,
                        duration_migr, 
                        tenants[i],
                        "config_xp3.properties",
                        migration_result_name,
                        faban_ips[i],
                        bpm_ips[i]) ]
                    run_command(command)
            else:
                if dur > 0:
                    command = ["./benchflow_onlytest_xp3.sh generate_test {0} {1} 5 {2} 0 {3} {4} {5} {6} {7}".format(
                        nb_agent,
                        bpm,
                        dur, 
                        tenants[i],
                        targets[i],
                        migration_result_name,
                        faban_ips[i],
                        bpm_ips[i]) ]                
                    print(command)
                    run_command(command)
        
                    
        # launch asynchronously not migrated tenants
        if dur > 0:
            commands_async = ["./benchflow_onlytest_xp3.sh execute_tests {0} {1} {2} {3}".format(
                migration_result_name,
                targets[i],
                tenants[i],
                faban_ips[i]
            ) for i in range(len(tenants)) if tenants[i] not in tenants_migr] 
            print(commands_async)
            processes = run_command_async(commands_async)     
        
        # launch synchronously migrated tenants
        if duration_migr > 0:
            commands = ["./benchflow_onlytest_xp3.sh execute_tests {0} {1} {2} {3}".format(
                migration_result_name,
                targets[i],
                tenants[i],
                faban_ips[i]
            ) for i in range(len(tenants)) if tenants[i] in tenants_migr]  
            run_command(commands)
            
            #wait for processes to terminate
            time.sleep(duration_before_migration)    
    
    # prepare migration for migrated tenants and migrate
    for i in range(len(tenants)):
        if tenants[i] in tenants_migr:
            result_agent_filename = "{0}_agent{1}_{2}_{3}.csv".format(migration_result_name, "pre", i, tenants[i])
            run_command("ssh -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@{0} docker cp $(docker ps -q --filter ancestor=bpmsagent):/Appagent/result.csv ./{1}".format(bpm_ips[i], result_agent_filename))
            run_command("scp -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@{0}:{1} {2}".format(bpm_ips[i], result_agent_filename, test_descriptor["path"]+result_agent_filename))
            result_agent_filename = "{0}_agentlog{1}_{2}_{3}.txt".format(migration_result_name, "pre", i, tenants[i])
            run_command("ssh -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@{0} docker cp $(docker ps -q --filter ancestor=bpmsagent):/Appagent/output_Agent.txt ./{1}".format(bpm_ips[i], result_agent_filename))
            run_command("scp -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@{0}:{1} {2}".format(bpm_ips[i], result_agent_filename, test_descriptor["path"]+result_agent_filename))
            
            run_command("docker service rm {0}_bpm-agent_{1}".format(migration_result_name, tenants[i]))
            date_premigr = datetime.datetime.now()
            if (back_and_forth is not None) and (back_and_forth > 0):
                for j in range(back_and_forth):
                    if duration_migr > 0:
                        res1 = run_command("./migrate.sh {0} {1}_migr_{2}".format(tenants[i], test_descriptor["path"] + migration_result_name, i))
                        res1 = run_command("./migrate2.sh {0} {1}_migr_{2}".format(tenants[i], test_descriptor["path"] + migration_result_name, i))
                        
                    else:
                        res1 = run_command("./migrate.sh {0} {1}_migr_{2} true".format(tenants[i], test_descriptor["path"] + migration_result_name, i))
                        res1 = run_command("./migrate2.sh {0} {1}_migr_{2} true".format(tenants[i], test_descriptor["path"] + migration_result_name, i))
                     
            else:
                if duration_migr > 0:
                    res1 = run_command("./migrate.sh {0} {1}_migr_{2}".format(tenants[i], test_descriptor["path"] + migration_result_name, i))
                else:
                    res1 = run_command("./migrate.sh {0} {1}_migr_{2} true".format(tenants[i], test_descriptor["path"] + migration_result_name, i))
                
            if (res1 !=0):
                print("Error in first migration.. skipping")
            date_postmigr = datetime.datetime.now()

            # if a migration is asked, and no back and forth (tenant stays on origin)
            if duration_migr > 0 and back_and_forth == 0:
                # relaunch faban agent on new
                run_command("docker service update --force {0}_faban-harness_{1}".format(migration_result_name, tenants[i]))
                run_command("docker service update --force {0}_faban-agent_{1}".format(migration_result_name, tenants[i]))
                time.sleep(10)
                with cd ("./test_executor"):
                    commands = ["./benchflow_onlytest_xp3.sh generate_test {0} {1} 5 {2} 0 {3} {4} {5} {6} {7}".format(
                        nb_agent,
                        bpm,
                        duration_migr, 
                        tenants[i],
                        "config2_xp3.properties",
                        migration_result_name,
                        faban_ips[i],
                        bpm_ips[i]) ]  
                    run_command(commands)


    # launch synchronously migrated tenants
    if duration_migr > 0 and back_and_forth == 0:
        with cd ("./test_executor"):
            commands = ["./benchflow_onlytest_xp3.sh execute_tests {0} {1} {2} {3}".format(
                migration_result_name,
                "config2_xp3.properties",
                tenants[i],
                faban_ips[i]
            ) for i in range(len(tenants)) if tenants[i] in tenants_migr]  
            run_command(commands)
            if len(tenants_migr) != len(tenants):
            # wait for other tenants to end
                wait_for_command_async(processes)

    
    for i in range(len(tenants)):
        result_agent_filename = "{0}_agent{1}_{2}_{3}.csv".format(migration_result_name, "post", i, tenants[i])
        run_command("ssh -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@{0} docker cp $(docker ps -q --filter ancestor=bpmsagent):/Appagent/result.csv ./{1}".format(bpm_ips[i], result_agent_filename))
        run_command("scp -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@{0}:{1} {2}".format(bpm_ips[i], result_agent_filename, test_descriptor["path"]+result_agent_filename))
        result_agent_filename = "{0}_agentlog{1}_{2}_{3}.txt".format(migration_result_name, "post", i, tenants[i])
        run_command("ssh -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@{0} docker cp $(docker ps -q --filter ancestor=bpmsagent):/Appagent/output_Agent.txt ./{1}".format(bpm_ips[i], result_agent_filename))
        run_command("scp -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@{0}:{1} {2}".format(bpm_ips[i], result_agent_filename, test_descriptor["path"]+result_agent_filename))
    

In [115]:
def create_test(test_descriptor):
    # initialize Swarm on infra & upload files
    import csv
    now=datetime.datetime.now(tzlocal())
    strnow = now.strftime("%Y%m%d%H%M%S")
    test_descriptor["ts_create_test"] = strnow

    if "back_and_forth" not in test_descriptor:
        test_descriptor["back_and_forth"] = None
    
    #generate agent conf
    agent_confs=[]

    for i in range(1,6):
        agent_conf={}
        agent_conf["tenant_name"] = "tenant{0}".format(i)
        agent_conf["login"] = "user1"
        agent_conf["pwd"] = "user1"
        agent_conf["filename"] = "config_tenant{0}.txt".format(i)
        agent_confs.append(agent_conf)

    migration_result_name = "{4}_m_{0}_{1}_{2}_{3}".format(test_descriptor["iteration"], test_descriptor["nb_agent"], test_descriptor["nb_process"],test_descriptor["bpm"].split("-")[0], test_descriptor["experiment_type"])
    path = "./results/{0}/".format(migration_result_name)
    import os
    if os.path.exists(path):
        print("{0} exists already, pass.".format(path))
        return None
    test_descriptor["migration_result_name"] = migration_result_name
    test_descriptor["path"]=path
    import pathlib
    pathlib.Path(path).mkdir(parents=True, exist_ok=True) 
    print ("Migration result name : " + migration_result_name)

    # init agent configuration
    for agent_conf in agent_confs:
        agent_conf_file = agent_conf["filename"]
        with open(test_descriptor["path"]+agent_conf_file, 'w') as csvfile:
            row = [agent_conf["tenant_name"], agent_conf["login"], agent_conf["pwd"], test_descriptor["nb_agent"], test_descriptor["nb_process"]]
            wr = csv.writer(csvfile, delimiter=';')
            wr.writerow(row)
        with open("./" + agent_conf_file, 'w') as csvfile:
            row = [agent_conf["tenant_name"], agent_conf["login"], agent_conf["pwd"], test_descriptor["nb_agent"], test_descriptor["nb_process"]]
            wr = csv.writer(csvfile, delimiter=';')
            wr.writerow(row)
    return test_descriptor

In [72]:
def deploy_needs(test_descriptor):
    now=datetime.datetime.now(tzlocal())
    strnow = now.strftime("%Y%m%d%H%M%S")
    test_descriptor["ts_deploy_needs"] = strnow
    
    tries = 3
    while tries >= 0:
        command = './init_swarm.sh'
        res = run_command(command)    
        if (res == 0):
            break
        else:
            print("Fail")
            tries = tries -1    

In [101]:
def deploy_bpms(test_descriptor):
    run_command("docker network rm my-overlay")    
    now=datetime.datetime.now(tzlocal())
    strnow = now.strftime("%Y%m%d%H%M%S")
    test_descriptor["ts_deploy_bpms"] = strnow

    import time
    run_command("./reset_docker.sh")
    time.sleep(10)
    # deploy BPMS
    tries = 3
    while tries >= 0:
        with cd ("./test_executor/templates/deployment_descriptor/template"):
            if test_descriptor["bpms"] == "bonita":
                res = run_command("docker stack deploy --compose-file=docker-compose.yml {0}".format(test_descriptor["migration_result_name"]))
            if (res != 0):
                tries = tries - 1
                time.sleep(10) 
                continue
            else:
                time.sleep(120) 
                break    
    # hack for address
    run_command("docker network create -d overlay my-overlay")

In [74]:
def launch_test(test_descriptor):
    now=datetime.datetime.now(tzlocal())
    strnow = now.strftime("%Y%m%d%H%M%S")
    test_descriptor["ts_launch_test"] = strnow

    
    if "duration_before_migration" not in test_descriptor:
        test_descriptor["duration_before_migration"] = 30
    #launch_tenant("pre", migration_result_name, "TestHumanTask-1.0.bar", dur, tenants, targets, 1, tenant_ips, bpm_ips, ["tenant3"], 60)
    launch_tenant("pre", 
                  test_descriptor["migration_result_name"], 
                  test_descriptor["bpm"], 
                  test_descriptor["duration_tenants_bg"], 
                  test_descriptor["tenants"], 
                  test_descriptor["targets"], 
                  test_descriptor["faban_agents"], 
                  test_descriptor["tenant_ips"], 
                  test_descriptor["bpm_ips"], 
                  test_descriptor["tenants_migr"], 
                  test_descriptor["duration_tenants_migr"],
                  test_descriptor["back_and_forth"],
                  test_descriptor["duration_before_migration"])


In [75]:
def copy_results(test_descriptor):
    now=datetime.datetime.now(tzlocal())
    strnow = now.strftime("%Y%m%d%H%M%S")
    test_descriptor["ts_copy_results"] = strnow

    for tenant in test_descriptor["tenants"]:
        print("Load tenant " + tenant)
        df = get_df_process("vm-vmonly-db-ref", tenant)
        if df is not None:
            df.to_csv(test_descriptor["path"]+"{0}_post_{1}.csv".format(test_descriptor["migration_result_name"], tenant))
            print("Ok")
    # copy faban + docker results
    for path in glob.glob("./test_executor/executed/{0}*".format(test_descriptor["migration_result_name"])):
        run_command(["mv {0} {1}".format(path, test_descriptor["path"])])    

In [117]:
def experiment(test_descriptor):
    test_descriptor = create_test(test_descriptor)
    if test_descriptor == None:
        return None

    if ("no_reset_needs" not in test_descriptor) or not(test_descriptor["no_reset_needs"] == "true"):
        deploy_needs(test_descriptor)

    if "no_reset_bpms" not in test_descriptor:
        run_command("./reset_docker.sh")
        deploy_bpms(test_descriptor)
    launch_test(test_descriptor)
    copy_results(test_descriptor)
    
    now=datetime.datetime.now(tzlocal())
    strnow = now.strftime("%Y%m%d%H%M%S")
    test_descriptor["ts_end_experiment"] = strnow
    with open(test_descriptor["path"]+'test_descriptor.csv', 'w') as f:
        for key in test_descriptor.keys():
            f.write("%s,%s\n"%(key,test_descriptor[key]))
    return test_descriptor

# XP3

In [ ]:
schemas=["TestHumanTask-1.0.bar", "M3Process--1.0.bar"]
nb_processes = [100, 200]
#schemas=["additionalApproval-2.0.bar"]
iterations = [11,12,13]
for nb_process in nb_processes:
    first = True
    for schema in schemas:
        for iteration in iterations:
            if first == True:
                test_descriptor = {
                    "bpm": schema,#"TestHumanTask-1.0.bar", # "additionalApproval-2.0.bar" #"M3Process--1.0.bar"
                    "bpms":"bonita",
                    "nb_agent":100,
                    "nb_process":nb_process,
                    "experiment_type":"xp3",
                    "duration_tenants_bg":1200,
                    "duration_tenants_migr":120,
                    "tenants":["tenant1","tenant2","tenant3","tenant4","tenant5"],
                    "tenants_migr":["tenant3"],
                    "targets":["config_pre.properties", "config_pre.properties", "config_pre.properties", "config_post.properties", "config_post.properties"],

                    "tenant_ips":["vm-vmonly-faban-agent1","vm-vmonly-faban-agent2","vm-vmonly-faban-agent3","vm-vmonly-faban-agent4", "vm-vmonly-faban-agent5"],
                    "bpm_ips":["vm-vmonly-bpm-agent1", "vm-vmonly-bpm-agent2", "vm-vmonly-bpm-agent3", "vm-vmonly-bpm-agent4", "vm-vmonly-bpm-agent5"],
                    "faban_agents":1,
                    "iteration":iteration,
                    "duration_before_migration":120
                }
                    
            else:
                test_descriptor = {
                    "bpm": schema,#"TestHumanTask-1.0.bar", # "additionalApproval-2.0.bar" #"M3Process--1.0.bar"
                    "bpms":"bonita",
                    "nb_agent":100,
                    "nb_process":nb_process,
                    "experiment_type":"xp3",
                    "duration_tenants_bg":1200,
                    "duration_tenants_migr":120,
                    "tenants":["tenant1","tenant2","tenant3","tenant4","tenant5"],
                    "tenants_migr":["tenant3"],
                    "targets":["config_pre.properties", "config_pre.properties", "config_pre.properties", "config_post.properties", "config_post.properties"],
                    "no_reset_needs":"true",

                    "tenant_ips":["vm-vmonly-faban-agent1","vm-vmonly-faban-agent2","vm-vmonly-faban-agent3","vm-vmonly-faban-agent4", "vm-vmonly-faban-agent5"],
                    "bpm_ips":["vm-vmonly-bpm-agent1", "vm-vmonly-bpm-agent2", "vm-vmonly-bpm-agent3", "vm-vmonly-bpm-agent4", "vm-vmonly-bpm-agent5"],
                    "faban_agents":1,
                    "iteration":iteration,
                    "duration_before_migration":120
            }
            experiment(test_descriptor)
            first = False

Migration result name : xp3_migr_11-100_100_100_TestHumanTask
./init_swarm.sh
./init_swarm.sh
Log 0 - 2018-07-20 00:37:11 : [ERROR]: No handlers could be found for logger
Log 0 - 2018-07-20 00:37:11 : "msrestazure.azure_active_directory"
Log 0 - 2018-07-20 00:37:11 : [DEPRECATION WARNING]: 'include' for playbook includes. You should use
Log 0 - 2018-07-20 00:37:11 : 'import_playbook' instead. This feature will be removed in version 2.8.
Log 0 - 2018-07-20 00:37:11 : Deprecation warnings can be disabled by setting deprecation_warnings=False in
Log 0 - 2018-07-20 00:37:11 : ansible.cfg.
Log 0 - 2018-07-20 00:37:11 : 
Log 0 - 2018-07-20 00:37:11 : PLAY [vmonly] ******************************************************************
Log 0 - 2018-07-20 00:37:12 : 
Log 0 - 2018-07-20 00:37:12 : TASK [Gathering Facts] *********************************************************
Log 0 - 2018-07-20 00:37:13 : ok: [vm-vmonly-bpm-agent3]
Log 0 - 2018-07-20 00:37:14 : ok: [vm-vmonly-bpm-agent1]
Log 0 - 20

Log 0 - 2018-07-20 00:37:42 : ok: [vm-vmonly-faban-agent2] => (item=[u'apt-transport-https', u'ca-certificates', u'curl', u'software-properties-common', u'linux-image-extra-virtual'])
Log 0 - 2018-07-20 00:37:42 : ok: [vm-vmonly-faban-agent3] => (item=[u'apt-transport-https', u'ca-certificates', u'curl', u'software-properties-common', u'linux-image-extra-virtual'])
Log 0 - 2018-07-20 00:37:42 : ok: [vm-vmonly-faban-agent4] => (item=[u'apt-transport-https', u'ca-certificates', u'curl', u'software-properties-common', u'linux-image-extra-virtual'])
Log 0 - 2018-07-20 00:37:42 : ok: [vm-vmonly-faban-agent5] => (item=[u'apt-transport-https', u'ca-certificates', u'curl', u'software-properties-common', u'linux-image-extra-virtual'])
Log 0 - 2018-07-20 00:37:43 : 
Log 0 - 2018-07-20 00:37:43 : TASK [docker : Add Docker apt key (alternative for older systems without SNI).] ***
Log 0 - 2018-07-20 00:37:43 : changed: [vm-vmonly-bpm-agent2]
Log 0 - 2018-07-20 00:37:44 : changed: [vm-vmonly-bpm-age

Log 0 - 2018-07-20 00:37:59 : ok: [vm-vmonly-bpm-agent4] => (item={u'version': u'1.21.1', u'name': u'urllib3[secure]'})
Log 0 - 2018-07-20 00:38:00 : ok: [vm-vmonly-bpms-from1] => (item={u'version': u'1.10.6', u'name': u'docker-py'})
Log 0 - 2018-07-20 00:38:00 : ok: [vm-vmonly-bpms-to1] => (item={u'version': u'1.10.6', u'name': u'docker-py'})
Log 0 - 2018-07-20 00:38:00 : ok: [vm-vmonly-db-ref] => (item={u'version': u'1.10.6', u'name': u'docker-py'})
Log 0 - 2018-07-20 00:38:00 : ok: [vm-vmonly-bpm-agent3] => (item={u'version': u'1.21.1', u'name': u'urllib3[secure]'})
Log 0 - 2018-07-20 00:38:00 : ok: [vm-vmonly-db-from1] => (item={u'version': u'1.10.6', u'name': u'docker-py'})
Log 0 - 2018-07-20 00:38:00 : ok: [vm-vmonly-db-ref] => (item={u'version': u'1.21.1', u'name': u'urllib3[secure]'})
Log 0 - 2018-07-20 00:38:00 : ok: [vm-vmonly-bpms-to1] => (item={u'version': u'1.21.1', u'name': u'urllib3[secure]'})
Log 0 - 2018-07-20 00:38:00 : ok: [vm-vmonly-bpms-from1] => (item={u'version':

Log 0 - 2018-07-20 00:38:15 : skipping: [vm-vmonly-faban-agent1]
Log 0 - 2018-07-20 00:38:15 : skipping: [vm-vmonly-faban-agent2]
Log 0 - 2018-07-20 00:38:15 : skipping: [vm-vmonly-faban-agent3]
Log 0 - 2018-07-20 00:38:15 : skipping: [vm-vmonly-faban-agent4]
Log 0 - 2018-07-20 00:38:15 : skipping: [vm-vmonly-faban-agent5]
Log 0 - 2018-07-20 00:38:15 : 
Log 0 - 2018-07-20 00:38:15 : PLAY [localhost] ***************************************************************
Log 0 - 2018-07-20 00:38:15 : 
Log 0 - 2018-07-20 00:38:15 : TASK [Gathering Facts] *********************************************************
Log 0 - 2018-07-20 00:38:16 : ok: [localhost]
Log 0 - 2018-07-20 00:38:16 : 
Log 0 - 2018-07-20 00:38:16 : TASK [List multiplexed SSH control sockets.] ***********************************
Log 0 - 2018-07-20 00:38:16 : changed: [localhost]
Log 0 - 2018-07-20 00:38:16 : 
Log 0 - 2018-07-20 00:38:16 : TASK [Exit multiplexed SSH.] ***************************************************
Log 0 - 20

Log 0 - 2018-07-20 00:38:59 : ok: [vm-vmonly-bpms-from1] => {
Log 0 - 2018-07-20 00:38:59 : "tags.bcd_type": "bonita"
Log 0 - 2018-07-20 00:38:59 : }
Log 0 - 2018-07-20 00:38:59 : ok: [vm-vmonly-bpms-to1] => {
Log 0 - 2018-07-20 00:38:59 : "tags.bcd_type": "bonita"
Log 0 - 2018-07-20 00:38:59 : }
Log 0 - 2018-07-20 00:38:59 : ok: [vm-vmonly-db-from1] => {
Log 0 - 2018-07-20 00:38:59 : "tags.bcd_type": "database"
Log 0 - 2018-07-20 00:38:59 : }
Log 0 - 2018-07-20 00:38:59 : ok: [vm-vmonly-db-ref] => {
Log 0 - 2018-07-20 00:38:59 : "tags.bcd_type": "database-ref"
Log 0 - 2018-07-20 00:38:59 : }
Log 0 - 2018-07-20 00:38:59 : ok: [vm-vmonly-db-to1] => {
Log 0 - 2018-07-20 00:38:59 : "tags.bcd_type": "database"
Log 0 - 2018-07-20 00:38:59 : }
Log 0 - 2018-07-20 00:38:59 : ok: [vm-vmonly-faban-agent1] => {
Log 0 - 2018-07-20 00:38:59 : "tags.bcd_type": "faban-agent"
Log 0 - 2018-07-20 00:38:59 : }
Log 0 - 2018-07-20 00:38:59 : ok: [vm-vmonly-faban-agent2] => {
Log 0 - 2018-07-20 00:38:59 : "

...

Log 0 - 2018-07-20 00:50:04 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-20 00:50:04 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-20 00:50:04 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-20 00:50:05 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-20 00:50:05 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-20 00:50:05 : overall progress: 1 out of 1 tasks
Log 0 - 2018-07-20 00:50:05 : verify: Waiting 5 seconds to verify that tasks are stable...
Log 0 - 2018-07-20 00:50:05 : verify: Waiting 5 seconds to verify that tasks are stable...
Log 0 - 2018-07-20 00:50:06 : verify: Waiting 5 seconds to verify that tasks are stable...
Log 0 - 2018-07-20 00:50:06 : verify: Waiting 5 seconds to verify that tasks are stable...
Log 0 - 2018-07-20 00:50:06 : verify: Waiting 5 seconds to verify that tasks are stable...
Log 0 - 2018-07-20 00:50:06 : verify: Waiting 4 seconds to verify that tasks are stable...
Log 0 - 2018-07-20 00:50:06 : verify: Waiting 4 seconds to verif

Log 0 - 2018-07-20 00:53:15 : Current STATUS: STARTED
Log 0 - 2018-07-20 00:53:31 : Current STATUS: STARTED
Log 0 - 2018-07-20 00:53:46 : Current STATUS: COMPLETED
Log 0 - 2018-07-20 00:53:47 : Data collection disabled:
Log 0 - 2018-07-20 00:53:47 : 
Log 0 - 2018-07-20 00:53:47 : Saving test results
Log 0 - 2018-07-20 00:53:49 : % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
Log 0 - 2018-07-20 00:53:49 : Dload  Upload   Total   Spent    Left  Speed
100  2201    0  2201    0     0   712k      0 --:--:-- --:--:-- --:--:-- 1074k-:-- --:--:-- --:--:--     0
Log 0 - 2018-07-20 00:53:49 : % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
Log 0 - 2018-07-20 00:53:49 : Dload  Upload   Total   Spent    Left  Speed
100 27440    0 27440    0     0  7897k      0 --:--:-- --:--:-- --:--:-- 8932k-:-- --:--:-- --:--:--     0
Log 0 - 2018-07-20 00:53:49 : % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
Log 0 - 201

Log 0 - 2018-07-20 00:54:00 : Current STATUS: STARTED
Log 1 - 2018-07-20 00:54:00 : Current STATUS: STARTED
Log 2 - 2018-07-20 00:54:00 : Current STATUS: STARTED
Log 3 - 2018-07-20 00:54:00 : Current STATUS: STARTED
Log 0 - 2018-07-20 00:54:15 : Current STATUS: STARTED
Log 1 - 2018-07-20 00:54:15 : Current STATUS: STARTED
Log 2 - 2018-07-20 00:54:15 : Current STATUS: STARTED
Log 3 - 2018-07-20 00:54:16 : Current STATUS: STARTED
Log 0 - 2018-07-20 00:54:31 : Current STATUS: STARTED
Log 1 - 2018-07-20 00:54:31 : Current STATUS: STARTED
Log 2 - 2018-07-20 00:54:31 : Current STATUS: STARTED
Log 3 - 2018-07-20 00:54:31 : Current STATUS: STARTED
Log 0 - 2018-07-20 00:54:47 : Current STATUS: STARTED
Log 1 - 2018-07-20 00:54:47 : Current STATUS: STARTED
Log 2 - 2018-07-20 00:54:47 : Current STATUS: STARTED
Log 3 - 2018-07-20 00:54:47 : Current STATUS: STARTED
Log 0 - 2018-07-20 00:55:02 : Current STATUS: STARTED
Log 1 - 2018-07-20 00:55:02 : Current STATUS: STARTED
Log 2 - 2018-07-20 00:55:02 

Log 0 - 2018-07-20 01:03:57 : Current STATUS: STARTED
Log 1 - 2018-07-20 01:03:57 : Current STATUS: STARTED
Log 2 - 2018-07-20 01:03:57 : Current STATUS: STARTED
Log 3 - 2018-07-20 01:03:59 : Current STATUS: STARTED
Log 0 - 2018-07-20 01:04:13 : Current STATUS: STARTED
Log 1 - 2018-07-20 01:04:13 : Current STATUS: STARTED
Log 2 - 2018-07-20 01:04:13 : Current STATUS: STARTED
Log 3 - 2018-07-20 01:04:14 : Current STATUS: STARTED
Log 0 - 2018-07-20 01:04:29 : Current STATUS: STARTED
Log 1 - 2018-07-20 01:04:29 : Current STATUS: STARTED
Log 2 - 2018-07-20 01:04:29 : Current STATUS: STARTED
Log 3 - 2018-07-20 01:04:30 : Current STATUS: STARTED
Log 0 - 2018-07-20 01:04:45 : Current STATUS: STARTED
Log 1 - 2018-07-20 01:04:45 : Current STATUS: STARTED
Log 2 - 2018-07-20 01:04:45 : Current STATUS: STARTED
Log 3 - 2018-07-20 01:04:46 : Current STATUS: STARTED
Log 0 - 2018-07-20 01:05:01 : Current STATUS: STARTED
Log 1 - 2018-07-20 01:05:01 : Current STATUS: STARTED
Log 2 - 2018-07-20 01:05:01 

scp -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@vm-vmonly-bpm-agent4:xp3_migr_11-100_100_100_TestHumanTask_agentpost_3_tenant4.csv ./results/xp3_migr_11-100_100_100_TestHumanTask/xp3_migr_11-100_100_100_TestHumanTask_agentpost_3_tenant4.csv
scp -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@vm-vmonly-bpm-agent4:xp3_migr_11-100_100_100_TestHumanTask_agentpost_3_tenant4.csv ./results/xp3_migr_11-100_100_100_TestHumanTask/xp3_migr_11-100_100_100_TestHumanTask_agentpost_3_tenant4.csv
ssh -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@vm-vmonly-bpm-agent5 docker cp $(docker ps -q --filter ancestor=bpmsagent):/Appagent/result.csv ./xp3_migr_11-100_100_100_TestHumanTask_agentpost_4_tenant5.csv
ssh -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@vm-vmonly-bpm-agent5 docker cp $(docker ps -q --filter ancestor=bpmsagent):/Appagent/result.csv ./xp3_migr_11-100_100_100_TestHumanTask_agentpost_4_tenant5.csv


# XP3'

In [138]:
schemas=["M3Process--1.0.bar","additionalApproval-2.0.bar", "TestHumanTask-1.0.bar"]
nb_processes = [0,50,100,200]
iterations = [34,35]
for nb_process in nb_processes:
    first = True
    for schema in schemas:
        for iteration in iterations:
            if first == True:
                test_descriptor = {
                    "bpm": schema,#"TestHumanTask-1.0.bar", # "additionalApproval-2.0.bar" #"M3Process--1.0.bar"
                    "bpms":"bonita",
                    "nb_agent":100,
                    "nb_process":nb_process,
                    "experiment_type":"xp3",
                    "duration_tenants_bg":1200,
                    "duration_tenants_migr":120,
                    "tenants":["tenant1","tenant2","tenant3"],
                    "tenants_migr":["tenant2"],
                    "targets":["config_pre.properties", "config_pre.properties", "config_post.properties"],

                    "tenant_ips":["vm-vmonly-faban-agent1","vm-vmonly-faban-agent2","vm-vmonly-faban-agent3"],
                    "bpm_ips":["vm-vmonly-bpm-agent1", "vm-vmonly-bpm-agent2", "vm-vmonly-bpm-agent3"],
                    "faban_agents":1,
                    "iteration":iteration,
                    "duration_before_migration":300
                }
                    
            else:
                test_descriptor = {
                    "bpm": schema,#"TestHumanTask-1.0.bar", # "additionalApproval-2.0.bar" #"M3Process--1.0.bar"
                    "bpms":"bonita",
                    "nb_agent":100,
                    "nb_process":nb_process,
                    "experiment_type":"xp3",
                    "duration_tenants_bg":1200,
                    "duration_tenants_migr":120,
                    "no_reset_needs":"true",
                    "tenants":["tenant1","tenant2","tenant3"],
                    "tenants_migr":["tenant2"],
                    "targets":["config_pre.properties", "config_pre.properties", "config_post.properties"],

                    "tenant_ips":["vm-vmonly-faban-agent1","vm-vmonly-faban-agent2","vm-vmonly-faban-agent3"],
                    "bpm_ips":["vm-vmonly-bpm-agent1", "vm-vmonly-bpm-agent2", "vm-vmonly-bpm-agent3"],
                    "faban_agents":1,
                    "iteration":iteration,
                    "duration_before_migration":300
            }
            test_descriptor = experiment(test_descriptor)
            if test_descriptor is not None:
                first = False

./results/xp3_m_34_100_0_M3Process/ exists already, pass.
./results/xp3_m_35_100_0_M3Process/ exists already, pass.
./results/xp3_m_34_100_0_additionalApproval/ exists already, pass.
./results/xp3_m_35_100_0_additionalApproval/ exists already, pass.
./results/xp3_m_34_100_0_TestHumanTask/ exists already, pass.
./results/xp3_m_35_100_0_TestHumanTask/ exists already, pass.
./results/xp3_m_34_100_50_M3Process/ exists already, pass.
./results/xp3_m_35_100_50_M3Process/ exists already, pass.
./results/xp3_m_34_100_50_additionalApproval/ exists already, pass.
./results/xp3_m_35_100_50_additionalApproval/ exists already, pass.
./results/xp3_m_34_100_50_TestHumanTask/ exists already, pass.
./results/xp3_m_35_100_50_TestHumanTask/ exists already, pass.
./results/xp3_m_34_100_100_M3Process/ exists already, pass.
./results/xp3_m_35_100_100_M3Process/ exists already, pass.
./results/xp3_m_34_100_100_additionalApproval/ exists already, pass.
./results/xp3_m_35_100_100_additionalApproval/ exists alr

Log 0 - 2018-07-24 22:50:39 : 
Log 0 - 2018-07-24 22:50:39 : TASK [docker : Add Docker apt key (alternative for older systems without SNI).] ***
Log 0 - 2018-07-24 22:50:40 : changed: [vm-vmonly-bpm-agent1]
Log 0 - 2018-07-24 22:50:40 : changed: [vm-vmonly-bpm-agent3]
Log 0 - 2018-07-24 22:50:40 : changed: [vm-vmonly-bpms-from1]
Log 0 - 2018-07-24 22:50:40 : changed: [vm-vmonly-bpms-to1]
Log 0 - 2018-07-24 22:50:40 : changed: [vm-vmonly-bpm-agent2]
Log 0 - 2018-07-24 22:50:41 : changed: [vm-vmonly-db-from1]
Log 0 - 2018-07-24 22:50:41 : changed: [vm-vmonly-db-ref]
Log 0 - 2018-07-24 22:50:41 : changed: [vm-vmonly-db-to1]
Log 0 - 2018-07-24 22:50:41 : changed: [vm-vmonly-faban-agent2]
Log 0 - 2018-07-24 22:50:41 : changed: [vm-vmonly-faban-agent1]
Log 0 - 2018-07-24 22:50:41 : changed: [vm-vmonly-faban-agent3]
Log 0 - 2018-07-24 22:50:41 : 
Log 0 - 2018-07-24 22:50:41 : TASK [docker : Add Docker repository.] *****************************************
Log 0 - 2018-07-24 22:50:42 : ok: [vm

Log 0 - 2018-07-24 22:50:56 : ok: [vm-vmonly-bpm-agent2]
Log 0 - 2018-07-24 22:50:56 : ok: [vm-vmonly-bpms-from1]
Log 0 - 2018-07-24 22:50:56 : ok: [vm-vmonly-bpm-agent3]
Log 0 - 2018-07-24 22:50:56 : ok: [vm-vmonly-bpm-agent1]
Log 0 - 2018-07-24 22:50:56 : ok: [vm-vmonly-bpms-to1]
Log 0 - 2018-07-24 22:50:56 : ok: [vm-vmonly-db-from1]
Log 0 - 2018-07-24 22:50:56 : ok: [vm-vmonly-db-ref]
Log 0 - 2018-07-24 22:50:56 : ok: [vm-vmonly-db-to1]
Log 0 - 2018-07-24 22:50:56 : ok: [vm-vmonly-faban-agent1]
Log 0 - 2018-07-24 22:50:56 : ok: [vm-vmonly-faban-agent2]
Log 0 - 2018-07-24 22:50:56 : ok: [vm-vmonly-faban-agent3]
Log 0 - 2018-07-24 22:50:56 : 
Log 0 - 2018-07-24 22:50:56 : TASK [docker : Ensure Docker daemon config file is present.] *******************
Log 0 - 2018-07-24 22:50:58 : ok: [vm-vmonly-bpm-agent1]
Log 0 - 2018-07-24 22:50:58 : ok: [vm-vmonly-bpm-agent3]
Log 0 - 2018-07-24 22:50:58 : ok: [vm-vmonly-bpm-agent2]
Log 0 - 2018-07-24 22:50:58 : ok: [vm-vmonly-bpms-from1]
Log 0 - 2

Log 0 - 2018-07-24 22:51:21 : ok: [vm-vmonly-bpm-agent1]
Log 0 - 2018-07-24 22:51:21 : ok: [vm-vmonly-bpm-agent2]
Log 0 - 2018-07-24 22:51:21 : ok: [vm-vmonly-bpm-agent3]
Log 0 - 2018-07-24 22:51:22 : ok: [vm-vmonly-bpms-from1]
Log 0 - 2018-07-24 22:51:22 : ok: [vm-vmonly-bpms-to1]
Log 0 - 2018-07-24 22:51:23 : ok: [vm-vmonly-db-from1]
Log 0 - 2018-07-24 22:51:23 : ok: [vm-vmonly-db-ref]
Log 0 - 2018-07-24 22:51:23 : ok: [vm-vmonly-db-to1]
Log 0 - 2018-07-24 22:51:23 : ok: [vm-vmonly-faban-agent2]
Log 0 - 2018-07-24 22:51:24 : ok: [vm-vmonly-faban-agent1]
Log 0 - 2018-07-24 22:51:24 : ok: [vm-vmonly-faban-agent3]
Log 0 - 2018-07-24 22:51:24 : 
Log 0 - 2018-07-24 22:51:24 : TASK [swarm-worker : quit swarm cluster] ***************************************
Log 0 - 2018-07-24 22:51:26 : changed: [vm-vmonly-bpm-agent3]
Log 0 - 2018-07-24 22:51:26 : changed: [vm-vmonly-bpm-agent1]
Log 0 - 2018-07-24 22:51:26 : changed: [vm-vmonly-bpm-agent2]
Log 0 - 2018-07-24 22:51:26 : changed: [vm-vmonly-b

Log 0 - 2018-07-24 22:51:56 : ok: [vm-vmonly-faban-agent1] => (item=bpm)
Log 0 - 2018-07-24 22:51:56 : ok: [vm-vmonly-faban-agent2] => (item=bpm)
Log 0 - 2018-07-24 22:51:57 : ok: [vm-vmonly-faban-agent3] => (item=bpm)
Log 0 - 2018-07-24 22:51:57 : 
Log 0 - 2018-07-24 22:51:57 : TASK [upload bpm to simulation instances] **************************************
Log 0 - 2018-07-24 22:51:57 : ok: [vm-vmonly-faban-agent1] => (item=/home/ubuntu/bpm/TestHumanTask-1.0.bar)
Log 0 - 2018-07-24 22:51:57 : ok: [vm-vmonly-faban-agent2] => (item=/home/ubuntu/bpm/TestHumanTask-1.0.bar)
Log 0 - 2018-07-24 22:51:57 : ok: [vm-vmonly-faban-agent3] => (item=/home/ubuntu/bpm/TestHumanTask-1.0.bar)
Log 0 - 2018-07-24 22:51:57 : ok: [vm-vmonly-faban-agent1] => (item=/home/ubuntu/bpm/Pool6--1.0.bar)
Log 0 - 2018-07-24 22:51:57 : ok: [vm-vmonly-faban-agent2] => (item=/home/ubuntu/bpm/Pool6--1.0.bar)
Log 0 - 2018-07-24 22:51:57 : ok: [vm-vmonly-faban-agent3] => (item=/home/ubuntu/bpm/Pool6--1.0.bar)
Log 0 - 2018

...

Log 0 - 2018-07-24 23:04:57 : DELETE FROM public.flownode_instance WHERE tenantid = 2;
Log 0 - 2018-07-24 23:04:57 : DELETE FROM public.report WHERE tenantid = 2;
Log 0 - 2018-07-24 23:04:57 : DELETE FROM public.processsupervisor WHERE tenantid = 2;
Log 0 - 2018-07-24 23:04:57 : DELETE FROM public.business_app WHERE tenantid = 2;
Log 0 - 2018-07-24 23:04:57 : DELETE FROM public.business_app_page WHERE tenantid = 2;
Log 0 - 2018-07-24 23:04:57 : DELETE FROM public.business_app_menu WHERE tenantid = 2;
Log 0 - 2018-07-24 23:04:57 : DELETE FROM public.command WHERE tenantid = 2;
Log 0 - 2018-07-24 23:04:57 : DELETE FROM public.data_instance WHERE tenantid = 2;
Log 0 - 2018-07-24 23:04:57 : DELETE FROM public.dependencymapping WHERE tenantid = 2;
Log 0 - 2018-07-24 23:04:57 : DELETE FROM public.dependency WHERE tenantid = 2;
Log 0 - 2018-07-24 23:04:57 : DELETE FROM public.external_identity_mapping WHERE tenantid = 2;
Log 0 - 2018-07-24 23:04:57 : DELETE FROM public.user_membership WHERE t

Log 0 - 2018-07-24 23:05:10 : 1/1: no suitable node (scheduling constraints not satisfied on 11 nodes; host-m…
Log 0 - 2018-07-24 23:05:10 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 23:05:10 : 1/1: no suitable node (scheduling constraints not satisfied on 11 nodes; host-m…
Log 0 - 2018-07-24 23:05:10 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 23:05:11 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 23:05:11 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 23:05:11 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 23:05:11 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 23:05:11 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 23:05:12 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 23:05:12 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 23:05:12 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 23:05:12 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 23:05:12 : overall progress: 1 out of 1 task

Log 0 - 2018-07-24 23:05:32 : verify: Waiting 5 seconds to verify that tasks are stable...
Log 0 - 2018-07-24 23:05:32 : verify: Waiting 5 seconds to verify that tasks are stable...
Log 0 - 2018-07-24 23:05:32 : verify: Waiting 4 seconds to verify that tasks are stable...
Log 0 - 2018-07-24 23:05:33 : verify: Waiting 4 seconds to verify that tasks are stable...
Log 0 - 2018-07-24 23:05:33 : verify: Waiting 4 seconds to verify that tasks are stable...
Log 0 - 2018-07-24 23:05:33 : verify: Waiting 4 seconds to verify that tasks are stable...
Log 0 - 2018-07-24 23:05:33 : verify: Waiting 4 seconds to verify that tasks are stable...
Log 0 - 2018-07-24 23:05:33 : verify: Waiting 3 seconds to verify that tasks are stable...
Log 0 - 2018-07-24 23:05:34 : verify: Waiting 3 seconds to verify that tasks are stable...
Log 0 - 2018-07-24 23:05:34 : verify: Waiting 3 seconds to verify that tasks are stable...
Log 0 - 2018-07-24 23:05:34 : verify: Waiting 3 seconds to verify that tasks are stable...

Log 0 - 2018-07-24 23:09:25 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:09:40 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:09:56 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:10:11 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:10:27 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:10:42 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:10:58 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:11:13 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:11:29 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:11:44 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:12:00 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:12:15 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:12:31 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:12:46 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:13:02 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:13:18 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:13:33 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:13:49 : Current STATUS: STARTED
Log 0 - 2018-07-24 23:14:04 

Load tenant tenant1
Load tenant tenant2
Ok
Load tenant tenant3
Ok
['mv ./test_executor/executed/xp3_m_35_100_200_additionalApproval_tenant2_LBNVZz4P ./results/xp3_m_35_100_200_additionalApproval/']
mv ./test_executor/executed/xp3_m_35_100_200_additionalApproval_tenant2_LBNVZz4P ./results/xp3_m_35_100_200_additionalApproval/
['mv ./test_executor/executed/xp3_m_35_100_200_additionalApproval_tenant3_D37Pn16q ./results/xp3_m_35_100_200_additionalApproval/']
mv ./test_executor/executed/xp3_m_35_100_200_additionalApproval_tenant3_D37Pn16q ./results/xp3_m_35_100_200_additionalApproval/
['mv ./test_executor/executed/xp3_m_35_100_200_additionalApproval_tenant2_vP7Aiehg ./results/xp3_m_35_100_200_additionalApproval/']
mv ./test_executor/executed/xp3_m_35_100_200_additionalApproval_tenant2_vP7Aiehg ./results/xp3_m_35_100_200_additionalApproval/
./results/xp3_m_34_100_200_TestHumanTask/ exists already, pass.
./results/xp3_m_35_100_200_TestHumanTask/ exists already, pass.


In [102]:
schemas=["TestHumanTask-1.0.bar"]
#schemas=["additionalApproval-2.0.bar", "TestHumanTask-1.0.bar","M3Process--1.0.bar"]
#nb_processes = [400]
nb_processes = [1000]
#schemas=["M3Process--1.0.bar"]
#iterations = [99]
iterations = [31]
for nb_process in nb_processes:
    first = True
    for schema in schemas:
        for iteration in iterations:
            if first == True:
                test_descriptor = {
                    "bpm": schema,#"TestHumanTask-1.0.bar", # "additionalApproval-2.0.bar" #"M3Process--1.0.bar"
                    "bpms":"bonita",
                    "nb_agent":100,
                    "nb_process":nb_process,
                    "experiment_type":"xp3",
                    "duration_tenants_bg":1200,
                    "duration_tenants_migr":120,
                    "tenants":["tenant1","tenant2","tenant3"],
                    "tenants_migr":["tenant2"],
                    "targets":["config_pre.properties", "config_pre.properties", "config_post.properties"],

                    "tenant_ips":["vm-vmonly-faban-agent1","vm-vmonly-faban-agent2","vm-vmonly-faban-agent3"],
                    "bpm_ips":["vm-vmonly-bpm-agent1", "vm-vmonly-bpm-agent2", "vm-vmonly-bpm-agent3"],
                    "faban_agents":1,
                    "iteration":iteration,
                    "duration_before_migration":300
                }
                    
            else:
                test_descriptor = {
                    "bpm": schema,#"TestHumanTask-1.0.bar", # "additionalApproval-2.0.bar" #"M3Process--1.0.bar"
                    "bpms":"bonita",
                    "nb_agent":100,
                    "nb_process":nb_process,
                    "experiment_type":"xp3",
                    "duration_tenants_bg":1200,
                    "duration_tenants_migr":120,
                    "no_reset_needs":"true",
                    "tenants":["tenant1","tenant2","tenant3"],
                    "tenants_migr":["tenant2"],
                    "targets":["config_pre.properties", "config_pre.properties", "config_post.properties"],

                    "tenant_ips":["vm-vmonly-faban-agent1","vm-vmonly-faban-agent2","vm-vmonly-faban-agent3"],
                    "bpm_ips":["vm-vmonly-bpm-agent1", "vm-vmonly-bpm-agent2", "vm-vmonly-bpm-agent3"],
                    "faban_agents":1,
                    "iteration":iteration,
                    "duration_before_migration":300
            }
            experiment(test_descriptor)
            first = False

Migration result name : xp3_m_31_100_0_additionalApproval
./init_swarm.sh
./init_swarm.sh
Log 0 - 2018-07-23 21:30:36 : [ERROR]: No handlers could be found for logger
Log 0 - 2018-07-23 21:30:36 : "msrestazure.azure_active_directory"
Log 0 - 2018-07-23 21:30:36 : [DEPRECATION WARNING]: 'include' for playbook includes. You should use
Log 0 - 2018-07-23 21:30:36 : 'import_playbook' instead. This feature will be removed in version 2.8.
Log 0 - 2018-07-23 21:30:36 : Deprecation warnings can be disabled by setting deprecation_warnings=False in
Log 0 - 2018-07-23 21:30:36 : ansible.cfg.
Log 0 - 2018-07-23 21:30:37 : 
Log 0 - 2018-07-23 21:30:37 : PLAY [vmonly] ******************************************************************
Log 0 - 2018-07-23 21:30:37 : 
Log 0 - 2018-07-23 21:30:37 : TASK [Gathering Facts] *********************************************************
Log 0 - 2018-07-23 21:30:38 : fatal: [vm-vmonly-bpm-agent4]: UNREACHABLE! => {"changed": false, "msg": "SSH Error: data could no

Log 0 - 2018-07-23 21:31:02 : changed: [vm-vmonly-bpms-to1]
Log 0 - 2018-07-23 21:31:02 : changed: [vm-vmonly-bpm-agent3]
Log 0 - 2018-07-23 21:31:02 : changed: [vm-vmonly-db-from1]
Log 0 - 2018-07-23 21:31:02 : changed: [vm-vmonly-db-ref]
Log 0 - 2018-07-23 21:31:02 : changed: [vm-vmonly-db-to1]
Log 0 - 2018-07-23 21:31:02 : changed: [vm-vmonly-faban-agent1]
Log 0 - 2018-07-23 21:31:02 : changed: [vm-vmonly-faban-agent2]
Log 0 - 2018-07-23 21:31:03 : changed: [vm-vmonly-faban-agent3]
Log 0 - 2018-07-23 21:31:03 : 
Log 0 - 2018-07-23 21:31:03 : TASK [docker : Add Docker repository.] *****************************************
Log 0 - 2018-07-23 21:31:04 : ok: [vm-vmonly-bpm-agent2]
Log 0 - 2018-07-23 21:31:04 : ok: [vm-vmonly-bpm-agent1]
Log 0 - 2018-07-23 21:31:04 : ok: [vm-vmonly-bpms-from1]
Log 0 - 2018-07-23 21:31:04 : ok: [vm-vmonly-bpms-to1]
Log 0 - 2018-07-23 21:31:04 : ok: [vm-vmonly-bpm-agent3]
Log 0 - 2018-07-23 21:31:04 : ok: [vm-vmonly-db-from1]
Log 0 - 2018-07-23 21:31:04 : 

Log 0 - 2018-07-23 21:31:18 : ok: [vm-vmonly-db-ref]
Log 0 - 2018-07-23 21:31:18 : ok: [vm-vmonly-db-to1]
Log 0 - 2018-07-23 21:31:18 : ok: [vm-vmonly-faban-agent1]
Log 0 - 2018-07-23 21:31:18 : ok: [vm-vmonly-faban-agent2]
Log 0 - 2018-07-23 21:31:18 : ok: [vm-vmonly-faban-agent3]
Log 0 - 2018-07-23 21:31:18 : 
Log 0 - 2018-07-23 21:31:18 : TASK [docker : Ensure Docker daemon config file is present.] *******************
Log 0 - 2018-07-23 21:31:19 : ok: [vm-vmonly-bpms-to1]
Log 0 - 2018-07-23 21:31:19 : ok: [vm-vmonly-bpm-agent2]
Log 0 - 2018-07-23 21:31:19 : ok: [vm-vmonly-bpm-agent1]
Log 0 - 2018-07-23 21:31:19 : ok: [vm-vmonly-bpms-from1]
Log 0 - 2018-07-23 21:31:19 : ok: [vm-vmonly-bpm-agent3]
Log 0 - 2018-07-23 21:31:20 : ok: [vm-vmonly-db-from1]
Log 0 - 2018-07-23 21:31:20 : ok: [vm-vmonly-db-ref]
Log 0 - 2018-07-23 21:31:20 : ok: [vm-vmonly-db-to1]
Log 0 - 2018-07-23 21:31:20 : ok: [vm-vmonly-faban-agent1]
Log 0 - 2018-07-23 21:31:20 : ok: [vm-vmonly-faban-agent2]
Log 0 - 2018-

Log 0 - 2018-07-23 21:31:45 : ok: [vm-vmonly-db-ref]
Log 0 - 2018-07-23 21:31:45 : ok: [vm-vmonly-db-to1]
Log 0 - 2018-07-23 21:31:45 : ok: [vm-vmonly-faban-agent1]
Log 0 - 2018-07-23 21:31:45 : ok: [vm-vmonly-faban-agent2]
Log 0 - 2018-07-23 21:31:46 : ok: [vm-vmonly-faban-agent3]
Log 0 - 2018-07-23 21:31:46 : 
Log 0 - 2018-07-23 21:31:46 : TASK [swarm-worker : quit swarm cluster] ***************************************
Log 0 - 2018-07-23 21:31:48 : changed: [vm-vmonly-bpm-agent2]
Log 0 - 2018-07-23 21:31:48 : changed: [vm-vmonly-bpm-agent1]
Log 0 - 2018-07-23 21:31:48 : changed: [vm-vmonly-bpm-agent3]
Log 0 - 2018-07-23 21:31:48 : changed: [vm-vmonly-bpms-from1]
Log 0 - 2018-07-23 21:31:49 : changed: [vm-vmonly-bpms-to1]
Log 0 - 2018-07-23 21:31:50 : changed: [vm-vmonly-db-from1]
Log 0 - 2018-07-23 21:31:50 : changed: [vm-vmonly-faban-agent1]
Log 0 - 2018-07-23 21:31:50 : changed: [vm-vmonly-db-to1]
Log 0 - 2018-07-23 21:31:50 : changed: [vm-vmonly-faban-agent2]
Log 0 - 2018-07-23 21

Log 0 - 2018-07-23 21:32:29 : ok: [vm-vmonly-faban-agent1] => (item=bpm)
Log 0 - 2018-07-23 21:32:30 : ok: [vm-vmonly-faban-agent2] => (item=bpm)
Log 0 - 2018-07-23 21:32:30 : ok: [vm-vmonly-faban-agent3] => (item=bpm)
Log 0 - 2018-07-23 21:32:30 : 
Log 0 - 2018-07-23 21:32:30 : TASK [upload bpm to simulation instances] **************************************
Log 0 - 2018-07-23 21:32:30 : ok: [vm-vmonly-faban-agent1] => (item=/home/ubuntu/bpm/TestHumanTask-1.0.bar)
Log 0 - 2018-07-23 21:32:30 : ok: [vm-vmonly-faban-agent2] => (item=/home/ubuntu/bpm/TestHumanTask-1.0.bar)
Log 0 - 2018-07-23 21:32:30 : ok: [vm-vmonly-faban-agent3] => (item=/home/ubuntu/bpm/TestHumanTask-1.0.bar)
Log 0 - 2018-07-23 21:32:30 : ok: [vm-vmonly-faban-agent1] => (item=/home/ubuntu/bpm/Pool6--1.0.bar)
Log 0 - 2018-07-23 21:32:30 : ok: [vm-vmonly-faban-agent2] => (item=/home/ubuntu/bpm/Pool6--1.0.bar)
Log 0 - 2018-07-23 21:32:30 : ok: [vm-vmonly-faban-agent3] => (item=/home/ubuntu/bpm/Pool6--1.0.bar)
Log 0 - 2018

...

Log 0 - 2018-07-23 22:18:36 : DELETE FROM public.processsupervisor WHERE tenantid = 3;
Log 0 - 2018-07-23 22:18:36 : DELETE FROM public.business_app WHERE tenantid = 3;
Log 0 - 2018-07-23 22:18:36 : DELETE FROM public.business_app_page WHERE tenantid = 3;
Log 0 - 2018-07-23 22:18:36 : DELETE FROM public.business_app_menu WHERE tenantid = 3;
Log 0 - 2018-07-23 22:18:36 : DELETE FROM public.command WHERE tenantid = 3;
Log 0 - 2018-07-23 22:18:36 : DELETE FROM public.data_instance WHERE tenantid = 3;
Log 0 - 2018-07-23 22:18:36 : DELETE FROM public.dependencymapping WHERE tenantid = 3;
Log 0 - 2018-07-23 22:18:36 : DELETE FROM public.dependency WHERE tenantid = 3;
Log 0 - 2018-07-23 22:18:36 : DELETE FROM public.external_identity_mapping WHERE tenantid = 3;
Log 0 - 2018-07-23 22:18:36 : DELETE FROM public.user_membership WHERE tenantid = 3;
Log 0 - 2018-07-23 22:18:36 : DELETE FROM public.custom_usr_inf_val WHERE tenantid = 3;
Log 0 - 2018-07-23 22:18:36 : DELETE FROM public.custom_usr_in

Log 0 - 2018-07-23 22:18:47 : 1/1: no suitable node (scheduling constraints not satisfied on 11 nodes; host-m…
Log 0 - 2018-07-23 22:18:47 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-23 22:18:47 : 1/1: no suitable node (scheduling constraints not satisfied on 11 nodes; host-m…
Log 0 - 2018-07-23 22:18:47 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-23 22:18:48 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-23 22:18:48 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-23 22:18:48 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-23 22:18:48 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-23 22:18:49 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-23 22:18:49 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-23 22:18:49 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-23 22:18:49 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-23 22:18:49 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-23 22:18:50 : overall progress: 1 out of 1 task

Log 0 - 2018-07-23 22:19:09 : verify: Waiting 5 seconds to verify that tasks are stable...
Log 0 - 2018-07-23 22:19:10 : verify: Waiting 5 seconds to verify that tasks are stable...
Log 0 - 2018-07-23 22:19:10 : verify: Waiting 5 seconds to verify that tasks are stable...
Log 0 - 2018-07-23 22:19:10 : verify: Waiting 4 seconds to verify that tasks are stable...
Log 0 - 2018-07-23 22:19:10 : verify: Waiting 4 seconds to verify that tasks are stable...
Log 0 - 2018-07-23 22:19:10 : verify: Waiting 4 seconds to verify that tasks are stable...
Log 0 - 2018-07-23 22:19:11 : verify: Waiting 4 seconds to verify that tasks are stable...
Log 0 - 2018-07-23 22:19:11 : verify: Waiting 4 seconds to verify that tasks are stable...
Log 0 - 2018-07-23 22:19:11 : verify: Waiting 3 seconds to verify that tasks are stable...
Log 0 - 2018-07-23 22:19:11 : verify: Waiting 3 seconds to verify that tasks are stable...
Log 0 - 2018-07-23 22:19:11 : verify: Waiting 3 seconds to verify that tasks are stable...

Log 0 - 2018-07-23 22:23:03 : Current STATUS: STARTED
Log 1 - 2018-07-23 22:23:03 : Current STATUS: STARTED
Log 0 - 2018-07-23 22:23:18 : Current STATUS: STARTED
Log 1 - 2018-07-23 22:23:19 : Current STATUS: STARTED
Log 0 - 2018-07-23 22:23:34 : Current STATUS: STARTED
Log 1 - 2018-07-23 22:23:34 : Current STATUS: STARTED
Log 0 - 2018-07-23 22:23:49 : Current STATUS: STARTED
Log 1 - 2018-07-23 22:23:50 : Current STATUS: STARTED
Log 0 - 2018-07-23 22:24:05 : Current STATUS: STARTED
Log 1 - 2018-07-23 22:24:05 : Current STATUS: STARTED
Log 0 - 2018-07-23 22:24:20 : Current STATUS: STARTED
Log 1 - 2018-07-23 22:24:21 : Current STATUS: STARTED
Log 0 - 2018-07-23 22:24:36 : Current STATUS: STARTED
Log 1 - 2018-07-23 22:24:36 : Current STATUS: STARTED
Log 0 - 2018-07-23 22:24:51 : Current STATUS: STARTED
Log 1 - 2018-07-23 22:24:52 : Current STATUS: STARTED
Log 0 - 2018-07-23 22:25:07 : Current STATUS: STARTED
Log 1 - 2018-07-23 22:25:07 : Current STATUS: STARTED
Log 0 - 2018-07-23 22:25:22 

scp -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@vm-vmonly-bpm-agent2:xp3_m_31_100_0_M3Process_agentlogpost_1_tenant2.txt ./results/xp3_m_31_100_0_M3Process/xp3_m_31_100_0_M3Process_agentlogpost_1_tenant2.txt
scp -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@vm-vmonly-bpm-agent2:xp3_m_31_100_0_M3Process_agentlogpost_1_tenant2.txt ./results/xp3_m_31_100_0_M3Process/xp3_m_31_100_0_M3Process_agentlogpost_1_tenant2.txt
ssh -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@vm-vmonly-bpm-agent3 docker cp $(docker ps -q --filter ancestor=bpmsagent):/Appagent/result.csv ./xp3_m_31_100_0_M3Process_agentpost_2_tenant3.csv
ssh -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@vm-vmonly-bpm-agent3 docker cp $(docker ps -q --filter ancestor=bpmsagent):/Appagent/result.csv ./xp3_m_31_100_0_M3Process_agentpost_2_tenant3.csv
scp -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@vm-vmonly-bpm-agent3:xp3_m_31_100_0_M3Process_agentpost_2_tenant3.csv ./results/xp3_m_31_100_0_M3Proc

# XP1

In [ ]:
#schemas=["TestHumanTask-1.0.bar", "additionalApproval-2.0.bar", "M3Process--1.0.bar"]
#durations=[0, 5, 10, 30, 60, 120, 300, 600]
schemas=["additionalApproval-2.0.bar"]
durations=[120, 300, 600]
for schema in schemas:
    for duration in durations:
        if duration == 0:
            test_descriptor = {
                "bpm": schema,#"TestHumanTask-1.0.bar", # "additionalApproval-2.0.bar" #"M3Process--1.0.bar"
                "bpms":"bonita",
                "nb_agent":1,
                "nb_process":99999,
                "experiment_type":"xp1",
                "duration_tenants_bg":0,
                "duration_tenants_migr":duration,
                "tenants":["tenant1"],
                "tenants_migr":["tenant1"],
                "targets":["config_pre.properties"],
                "tenant_ips":["vm-vmonly-faban-agent1"],
                "bpm_ips":["vm-vmonly-bpm-agent1"],
                "faban_agents":1,
                "iteration":duration,
                "back_and_forth":3
            }            
        else:
            test_descriptor = {
                "bpm": schema,#"TestHumanTask-1.0.bar", # "additionalApproval-2.0.bar" #"M3Process--1.0.bar"
                "bpms":"bonita",
                "nb_agent":1,
                "nb_process":99999,
                "experiment_type":"xp1",
                "duration_tenants_bg":0,
                "duration_tenants_migr":duration,
                "tenants":["tenant1"],
                "tenants_migr":["tenant1"],
                "targets":["config_pre.properties"],              
                "tenant_ips":["vm-vmonly-faban-agent1"],
                "bpm_ips":["vm-vmonly-bpm-agent1"],
                "faban_agents":1,
                "no_reset_needs":"true",
                "iteration":duration,
                "back_and_forth":3
            }
        experiment(test_descriptor) 
            

# XP2

In [ ]:
schemas=["M3Process--1.0.bar","additionalApproval-2.0.bar", "TestHumanTask-1.0.bar"]
iterations=[11,12,13,14,15]
not_first="false"
for schema in schemas:
    for iteration in iterations:
        test_descriptor = {
            "bpm": schema,#"TestHumanTask-1.0.bar", # "additionalApproval-2.0.bar" #"M3Process--1.0.bar"
            "bpms":"bonita",
            "nb_agent":100,
            "nb_process":0,
            "experiment_type":"xp2",
            "duration_tenants_bg":0,
            "duration_tenants_migr":600,
            "tenants":["tenant1"],
            "tenants_migr":["tenant1"],
            "targets":["config_pre.properties"],              
            "tenant_ips":["vm-vmonly-faban-agent1"],
            "bpm_ips":["vm-vmonly-bpm-agent1"],
            "faban_agents":1,
            "iteration":iteration,
            "duration_before_migration":300,
            "no_reset_needs":not_first,
        }
        experiment(test_descriptor) 
        not_first="true"

In [110]:
schemas=["additionalApproval-2.0.bar"]
iterations=[11]
not_first="false"
for schema in schemas:
    for iteration in iterations:
        test_descriptor = {
            "bpm": schema,#"TestHumanTask-1.0.bar", # "additionalApproval-2.0.bar" #"M3Process--1.0.bar"
            "bpms":"bonita",
            "nb_agent":100,
            "nb_process":0,
            "experiment_type":"xp2",
            "duration_tenants_bg":0,
            "duration_tenants_migr":600,
            "tenants":["tenant1"],
            "tenants_migr":["tenant1"],
            "targets":["config_pre.properties"],              
            "tenant_ips":["vm-vmonly-faban-agent1"],
            "bpm_ips":["vm-vmonly-bpm-agent1"],
            "faban_agents":1,
            "iteration":iteration,
            "duration_before_migration":300,
            "no_reset_needs":not_first,
        }
        experiment(test_descriptor) 
        not_first="true"

Migration result name : xp2_m_11_100_0_additionalApproval
./init_swarm.sh
./init_swarm.sh
Log 0 - 2018-07-24 00:08:18 : [ERROR]: No handlers could be found for logger
Log 0 - 2018-07-24 00:08:18 : "msrestazure.azure_active_directory"
Log 0 - 2018-07-24 00:08:18 : [DEPRECATION WARNING]: 'include' for playbook includes. You should use
Log 0 - 2018-07-24 00:08:18 : 'import_playbook' instead. This feature will be removed in version 2.8.
Log 0 - 2018-07-24 00:08:18 : Deprecation warnings can be disabled by setting deprecation_warnings=False in
Log 0 - 2018-07-24 00:08:18 : ansible.cfg.
Log 0 - 2018-07-24 00:08:19 : 
Log 0 - 2018-07-24 00:08:19 : PLAY [vmonly] ******************************************************************
Log 0 - 2018-07-24 00:08:19 : 
Log 0 - 2018-07-24 00:08:19 : TASK [Gathering Facts] *********************************************************
Log 0 - 2018-07-24 00:08:19 : fatal: [vm-vmonly-bpm-agent5]: UNREACHABLE! => {"changed": false, "msg": "SSH Error: data could no

Log 0 - 2018-07-24 00:08:35 : changed: [vm-vmonly-bpms-to1]
Log 0 - 2018-07-24 00:08:35 : changed: [vm-vmonly-bpms-from1]
Log 0 - 2018-07-24 00:08:35 : changed: [vm-vmonly-db-from1]
Log 0 - 2018-07-24 00:08:35 : changed: [vm-vmonly-db-ref]
Log 0 - 2018-07-24 00:08:35 : changed: [vm-vmonly-db-to1]
Log 0 - 2018-07-24 00:08:35 : changed: [vm-vmonly-faban-agent1]
Log 0 - 2018-07-24 00:08:35 : changed: [vm-vmonly-faban-agent2]
Log 0 - 2018-07-24 00:08:35 : changed: [vm-vmonly-faban-agent3]
Log 0 - 2018-07-24 00:08:36 : 
Log 0 - 2018-07-24 00:08:36 : TASK [docker : Add Docker repository.] *****************************************
Log 0 - 2018-07-24 00:08:36 : ok: [vm-vmonly-bpm-agent2]
Log 0 - 2018-07-24 00:08:36 : ok: [vm-vmonly-bpm-agent3]
Log 0 - 2018-07-24 00:08:36 : ok: [vm-vmonly-bpm-agent1]
Log 0 - 2018-07-24 00:08:37 : ok: [vm-vmonly-bpms-from1]
Log 0 - 2018-07-24 00:08:37 : ok: [vm-vmonly-bpms-to1]
Log 0 - 2018-07-24 00:08:37 : ok: [vm-vmonly-db-from1]
Log 0 - 2018-07-24 00:08:37 : 

Log 0 - 2018-07-24 00:08:50 : ok: [vm-vmonly-bpm-agent2]
Log 0 - 2018-07-24 00:08:51 : ok: [vm-vmonly-db-from1]
Log 0 - 2018-07-24 00:08:51 : ok: [vm-vmonly-db-ref]
Log 0 - 2018-07-24 00:08:51 : ok: [vm-vmonly-db-to1]
Log 0 - 2018-07-24 00:08:51 : ok: [vm-vmonly-faban-agent1]
Log 0 - 2018-07-24 00:08:51 : ok: [vm-vmonly-faban-agent2]
Log 0 - 2018-07-24 00:08:51 : ok: [vm-vmonly-faban-agent3]
Log 0 - 2018-07-24 00:08:51 : 
Log 0 - 2018-07-24 00:08:51 : TASK [docker : Ensure Docker daemon config file is present.] *******************
Log 0 - 2018-07-24 00:08:52 : ok: [vm-vmonly-bpm-agent3]
Log 0 - 2018-07-24 00:08:52 : ok: [vm-vmonly-bpm-agent2]
Log 0 - 2018-07-24 00:08:52 : ok: [vm-vmonly-bpm-agent1]
Log 0 - 2018-07-24 00:08:52 : ok: [vm-vmonly-bpms-from1]
Log 0 - 2018-07-24 00:08:52 : ok: [vm-vmonly-bpms-to1]
Log 0 - 2018-07-24 00:08:53 : ok: [vm-vmonly-db-from1]
Log 0 - 2018-07-24 00:08:53 : ok: [vm-vmonly-db-ref]
Log 0 - 2018-07-24 00:08:53 : ok: [vm-vmonly-faban-agent1]
Log 0 - 2018-

Log 0 - 2018-07-24 00:09:19 : ok: [vm-vmonly-db-from1]
Log 0 - 2018-07-24 00:09:19 : ok: [vm-vmonly-db-ref]
Log 0 - 2018-07-24 00:09:19 : ok: [vm-vmonly-db-to1]
Log 0 - 2018-07-24 00:09:19 : ok: [vm-vmonly-faban-agent1]
Log 0 - 2018-07-24 00:09:19 : ok: [vm-vmonly-faban-agent2]
Log 0 - 2018-07-24 00:09:20 : ok: [vm-vmonly-faban-agent3]
Log 0 - 2018-07-24 00:09:20 : 
Log 0 - 2018-07-24 00:09:20 : TASK [swarm-worker : quit swarm cluster] ***************************************
Log 0 - 2018-07-24 00:09:22 : changed: [vm-vmonly-bpms-from1]
Log 0 - 2018-07-24 00:09:22 : changed: [vm-vmonly-bpm-agent2]
Log 0 - 2018-07-24 00:09:22 : changed: [vm-vmonly-bpm-agent1]
Log 0 - 2018-07-24 00:09:22 : changed: [vm-vmonly-bpm-agent3]
Log 0 - 2018-07-24 00:09:23 : changed: [vm-vmonly-bpms-to1]
Log 0 - 2018-07-24 00:09:24 : changed: [vm-vmonly-db-from1]
Log 0 - 2018-07-24 00:09:24 : changed: [vm-vmonly-db-ref]
Log 0 - 2018-07-24 00:09:24 : changed: [vm-vmonly-faban-agent1]
Log 0 - 2018-07-24 00:09:24 : 

Log 0 - 2018-07-24 00:09:54 : ok: [vm-vmonly-faban-agent1] => (item=bpm)
Log 0 - 2018-07-24 00:09:54 : ok: [vm-vmonly-faban-agent2] => (item=bpm)
Log 0 - 2018-07-24 00:09:54 : ok: [vm-vmonly-faban-agent3] => (item=bpm)
Log 0 - 2018-07-24 00:09:54 : 
Log 0 - 2018-07-24 00:09:54 : TASK [upload bpm to simulation instances] **************************************
Log 0 - 2018-07-24 00:09:55 : ok: [vm-vmonly-faban-agent1] => (item=/home/ubuntu/bpm/TestHumanTask-1.0.bar)
Log 0 - 2018-07-24 00:09:55 : ok: [vm-vmonly-faban-agent2] => (item=/home/ubuntu/bpm/TestHumanTask-1.0.bar)
Log 0 - 2018-07-24 00:09:55 : ok: [vm-vmonly-faban-agent3] => (item=/home/ubuntu/bpm/TestHumanTask-1.0.bar)
Log 0 - 2018-07-24 00:09:55 : ok: [vm-vmonly-faban-agent1] => (item=/home/ubuntu/bpm/Pool6--1.0.bar)
Log 0 - 2018-07-24 00:09:55 : ok: [vm-vmonly-faban-agent2] => (item=/home/ubuntu/bpm/Pool6--1.0.bar)
Log 0 - 2018-07-24 00:09:55 : ok: [vm-vmonly-faban-agent3] => (item=/home/ubuntu/bpm/Pool6--1.0.bar)
Log 0 - 2018

...

100 21.0M    0 21.0M    0     0  69.6M      0 --:--:-- --:--:-- --:--:-- 69.8M-:-- --:--:-- --:--:--     0
ssh -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@vm-vmonly-bpm-agent1 docker cp $(docker ps -q --filter ancestor=bpmsagent):/Appagent/result.csv ./xp2_m_11_100_0_additionalApproval_agentpre_0_tenant1.csv
ssh -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@vm-vmonly-bpm-agent1 docker cp $(docker ps -q --filter ancestor=bpmsagent):/Appagent/result.csv ./xp2_m_11_100_0_additionalApproval_agentpre_0_tenant1.csv
scp -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@vm-vmonly-bpm-agent1:xp2_m_11_100_0_additionalApproval_agentpre_0_tenant1.csv ./results/xp2_m_11_100_0_additionalApproval/xp2_m_11_100_0_additionalApproval_agentpre_0_tenant1.csv
scp -o CheckHostIP=no -o StrictHostKeyChecking=no ubuntu@vm-vmonly-bpm-agent1:xp2_m_11_100_0_additionalApproval_agentpre_0_tenant1.csv ./results/xp2_m_11_100_0_additionalApproval/xp2_m_11_100_0_additionalApproval_agentpre_0_tenant1.c

Log 0 - 2018-07-24 00:40:35 : insert into psql_target.page_mapping select * FROM public.page_mapping WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:35 : insert into psql_target.form_mapping select * FROM public.form_mapping WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:35 : insert into psql_target.theme select * FROM public.theme WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:35 : insert into psql_target.job_desc select * FROM public.job_desc WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:35 : insert into psql_target.job_param select * FROM public.job_param WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:35 : insert into psql_target.job_log select * FROM public.job_log WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:35 : insert into psql_target.profile select * FROM public.profile WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:35 : insert into psql_target.profileentry select * FROM public.profileentry WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:35 : insert into psql_target.profilemember select * FRO

Log 0 - 2018-07-24 00:40:38 : DELETE FROM public.profilemember WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:38 : DELETE FROM public.profileentry WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:38 : DELETE FROM public.profile WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:38 : DELETE FROM public.job_log WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:38 : DELETE FROM public.job_param WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:38 : DELETE FROM public.job_desc WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:38 : DELETE FROM public.theme WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:39 : DELETE FROM public.form_mapping WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:39 : DELETE FROM public.page_mapping WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:39 : DELETE FROM public.process_content WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:39 : DELETE FROM public.proc_parameter WHERE tenantid = 2;
Log 0 - 2018-07-24 00:40:39 : 2018-07-24 00:40:39.611271 - activating tenant 2 on http://vm-vmonly-bpms-to1:8081/bon

Log 0 - 2018-07-24 00:41:46 : 1/1: no suitable node (scheduling constraints not satisfied on 11 nodes; host-m…
Log 0 - 2018-07-24 00:41:46 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 00:41:47 : 1/1: no suitable node (scheduling constraints not satisfied on 11 nodes; host-m…
Log 0 - 2018-07-24 00:41:47 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 00:41:47 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 00:41:47 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 00:41:47 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 00:41:47 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 00:41:48 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 00:41:48 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 00:41:48 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 00:41:48 : overall progress: 0 out of 1 tasks
Log 0 - 2018-07-24 00:41:49 : overall progress: 1 out of 1 tasks
Log 0 - 2018-07-24 00:41:49 : verify: Waiting 5 seconds to veri

KeyboardInterrupt: 

# RESET 

In [137]:
run_command("./reset_docker.sh")


./reset_docker.sh
./reset_docker.sh
Log 0 - 2018-07-24 22:49:25 : "docker stack rm" requires at least 1 argument.
Log 0 - 2018-07-24 22:49:25 : See 'docker stack rm --help'.
Log 0 - 2018-07-24 22:49:25 : 
Log 0 - 2018-07-24 22:49:25 : Usage:  docker stack rm STACK [STACK...] [flags]
Log 0 - 2018-07-24 22:49:25 : 
Log 0 - 2018-07-24 22:49:25 : Remove one or more stacks


1

In [89]:
run_command("rm -Rf ./test_executor/to_execute/*")

rm -Rf ./test_executor/to_execute/*
rm -Rf ./test_executor/to_execute/*


0

In [99]:
copy_results(test_descriptor)

Load tenant tenant1
Ok
Load tenant tenant2
Ok
Load tenant tenant3
Ok
['mv ./test_executor/executed/xp3_m_31_100_0_M3Process_tenant2_FHYfbXAX ./results/xp3_m_31_100_0_M3Process/']
mv ./test_executor/executed/xp3_m_31_100_0_M3Process_tenant2_FHYfbXAX ./results/xp3_m_31_100_0_M3Process/
['mv ./test_executor/executed/xp3_m_31_100_0_M3Process_tenant3_zMoX9m3r ./results/xp3_m_31_100_0_M3Process/']
mv ./test_executor/executed/xp3_m_31_100_0_M3Process_tenant3_zMoX9m3r ./results/xp3_m_31_100_0_M3Process/
['mv ./test_executor/executed/xp3_m_31_100_0_M3Process_tenant2_90BiMims ./results/xp3_m_31_100_0_M3Process/']
mv ./test_executor/executed/xp3_m_31_100_0_M3Process_tenant2_90BiMims ./results/xp3_m_31_100_0_M3Process/
